<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=424f17ff4f4a739d252bb632f4136b0f79316da1e6426287189f594326b5a6f2
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [2]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-1000)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [3]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv')

#mypf = mypf[mypf['Nifty100'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_ids = stock_n100

#mypf = mypf[mypf.Forecast.notnull()]
#stock_ids = mypf['Symbol'].unique()

stock_ids.sort()

Mounted at /content/drive


In [4]:
# star stocks & my stocks
star_stocks = list(mypf[mypf['StarStock'] == 1]['Stock'].values)
my_stocks = list(mypf[mypf['InPortfolio'] == 1]['Symbol'].values)

In [5]:
# update Nifty features
'''
stock = 'INFY'
column = 'Conviction'
value = 'H'
nifty100_df.loc[nifty100_df['Symbol'] == stock, column] = value

mypf.to_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv',index=False)
'''
mypf['Conviction'].value_counts()

M    67
L    63
H    37
Name: Conviction, dtype: int64

In [6]:
# star stocks
mypf['StarStock'].value_counts()

0    105
1     62
Name: StarStock, dtype: int64

In [7]:
# v80 stocks
mypf['v80'].value_counts()

0    98
1    69
Name: v80, dtype: int64

In [8]:
# latest quarter
mypf['LatestQtr'].value_counts()

1    85
0    82
Name: LatestQtr, dtype: int64

In [9]:
# star + v80 stocks
mypf[mypf['v80'] == 1]['StarStock'].value_counts()

0    44
1    25
Name: StarStock, dtype: int64

In [10]:
# stocks in portfolio
mypf['InPortfolio'].value_counts()

0    144
1     23
Name: InPortfolio, dtype: int64

In [11]:
# import libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()
import pandas_ta as ta

In [12]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_pe(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()  
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")        
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:
    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
  except ValueError as ve:
    current_pe = 1000 

  return current_pe


In [13]:
# function to compute stock attributes

def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'
    
    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = yfin.Ticker(stock_symbol).history(period='3y')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    #stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    #stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    #print(stock_df)

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close'].rolling(window = short_window, min_periods = 1).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close'].rolling(window = mid_window, min_periods = 1).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df['200_SMA_Lag10'] = stock_df['200_SMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_SMA'] - stock_df['200_SMA_Lag10'])*100/stock_df['200_SMA'],2)
    stock_df['50_SMA_Lag5'] = stock_df['50_SMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_SMA'] - stock_df['50_SMA_Lag5'])*100/stock_df['50_SMA'],2)
    
    #stock_df.ta.rsi(append=True)
    #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
    stock_df['Close'] = round(stock_df['Close'],0)    
    #stock_df['RSI'] = round(stock_df['RSI'],0) 
    stock_df['Max'] = round(max(stock_df['Close']),0)
    stock_df['Min'] = round(min(stock_df['Close']),0)   
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Upside%ATH'] = round((stock_df['Max']-stock_df['Close'])*100/(stock_df['Close']),0)
    stock_df.drop(['Open', 'Low', 'High', 'Volume','20_SMA','50_SMA','200_SMA','200_SMA_Lag10','50_SMA_Lag5'
                   ], axis=1, inplace=True)   
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    #print(stock_id)
    if stock_id in my_stocks:
      current_pe = get_current_pe(stock_id)
    else:
      current_pe = 1000

    stock_id = stock_id.upper() + '.NS'    
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Avg_Cost'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['AvgCost'].values[0]
    tmp['Shares'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Shares'].values[0]
    tmp['Med_PE'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['MedianPE'].values[0]    
    tmp['Curr_PE'] = current_pe
    tmp['BizOutlook'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizOutlook'].values[0] 
    tmp['BizDurability'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['BizDurability'].values[0]   
    tmp['Conviction'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Conviction'].values[0]    
    tmp['LatestQtr'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['LatestQtr'].values[0]  
    tmp['StarStock'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['StarStock'].values[0]       
    tmp['v80'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['v80'].values[0] 
    df_prec_dev = df_prec_dev.append(tmp)

In [14]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,
2023-04-18 00:00:00+05:30,17660.0,^NSEI,0.47,0.59,-0.02,18812.0,10806.0,14.39,7.0
2023-04-25 00:00:00+05:30,17769.0,^NSEI,0.82,0.58,-0.02,18812.0,10806.0,13.03,6.0
2023-05-03 00:00:00+05:30,18090.0,^NSEI,2.35,0.55,0.08,18812.0,10806.0,9.02,4.0
2023-05-10 00:00:00+05:30,18315.0,^NSEI,3.30,0.59,0.17,18812.0,10806.0,6.21,3.0
2023-05-17 00:00:00+05:30,18182.0,^NSEI,2.29,0.56,0.48,18812.0,10806.0,7.87,3.0


In [15]:
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)
df_prec_dev.drop(['Date'], axis=1, inplace=True)

mypf['Stock'] = mypf['Symbol'] + '.NS'
#df_prec_dev.drop(['Med_PE'], axis=1, inplace=True)

In [16]:
df_prec_dev['Discount%PE'] = round((df_prec_dev['Med_PE']-df_prec_dev['Curr_PE'])*100/(df_prec_dev['Med_PE']),0)
#df_prec_dev['Upside%Estimate'] = round((df_prec_dev['Max']-df_prec_dev['Avg_Cost'])*100/(df_prec_dev['Avg_Cost']),0)
df_prec_dev = df_prec_dev.set_index('Stock')

In [17]:
# portfolio stocks
cols = ['Close',	'Dev%_200',	'Slope%_200',	'Slope%_50', 'Discount%',	'Upside%ATH',	'Avg_Cost',	'Med_PE',	'Curr_PE',	'Conviction', 'Discount%PE', 'LatestQtr', 'StarStock', 'v80']
tmp_df = df_prec_dev[cols]
tmp_df = tmp_df[tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 23


,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
VALIANTORG.NS,475.0,-17.64,-1.04,1.31,93.54,290.0,481.73,24.0,14.9,M,38.0,0,0,0
GLAND.NS,1339.0,-22.03,-3.09,0.39,94.15,222.0,1354.78,45.2,22.3,M,51.0,0,0,0
LUXIND.NS,1473.0,-7.22,-1.26,1.21,89.93,209.0,1499.87,21.4,24.5,M,-14.0,0,0,0
VAIBHAVGBL.NS,320.0,-0.39,0.00,-0.32,92.95,202.0,320.64,35.4,50.3,M,-42.0,0,0,0
INDIGOPNTS.NS,1328.0,1.18,-0.46,2.61,83.83,134.0,1215.89,104.9,53.6,M,49.0,0,1,0
JUBLFOOD.NS,481.0,-8.19,-0.95,0.45,82.91,88.0,468.21,90.5,89.9,H,1.0,0,1,0
WIPRO.NS,382.0,-3.22,-0.25,-0.27,74.05,87.0,382.90,20.9,18.5,L,11.0,1,0,0
IRCTC.NS,623.0,-5.93,0.30,0.33,59.65,87.0,629.08,77.4,52.8,H,32.0,0,1,0
HDFCAMC.NS,1817.0,-7.61,-0.25,0.17,90.28,78.0,1856.05,38.5,27.2,M,29.0,1,0,1


In [18]:
# top 7 from portfolio for SIP
tmp_df.sort_values(by = 'Upside%ATH', ascending=False).head(7)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
VALIANTORG.NS,475.0,-17.64,-1.04,1.31,93.54,290.0,481.73,24.0,14.9,M,38.0,0,0,0
GLAND.NS,1339.0,-22.03,-3.09,0.39,94.15,222.0,1354.78,45.2,22.3,M,51.0,0,0,0
LUXIND.NS,1473.0,-7.22,-1.26,1.21,89.93,209.0,1499.87,21.4,24.5,M,-14.0,0,0,0
VAIBHAVGBL.NS,320.0,-0.39,0.00,-0.32,92.95,202.0,320.64,35.4,50.3,M,-42.0,0,0,0
INDIGOPNTS.NS,1328.0,1.18,-0.46,2.61,83.83,134.0,1215.89,104.9,53.6,M,49.0,0,1,0
JUBLFOOD.NS,481.0,-8.19,-0.95,0.45,82.91,88.0,468.21,90.5,89.9,H,1.0,0,1,0
WIPRO.NS,382.0,-3.22,-0.25,-0.27,74.05,87.0,382.90,20.9,18.5,L,11.0,1,0,0


In [19]:
# top 7 from portfolio to book profits
tmp_df.sort_values(by = 'Upside%ATH', ascending=True).head(7)

,Close,Dev%_200,Slope%_200,Slope%_50,Discount%,Upside%ATH,Avg_Cost,Med_PE,Curr_PE,Conviction,Discount%PE,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
EICHERMOT.NS,3629.0,9.30,0.72,1.29,11.97,6.0,3336.21,44.7,34.1,H,24.0,1,1,1
TATACONSUM.NS,778.0,1.73,0.00,0.96,23.32,12.0,758.64,68.8,67.1,H,2.0,1,1,0
PIDILITIND.NS,2545.0,0.16,0.43,1.17,23.58,14.0,2442.09,94.6,102.0,H,-8.0,1,1,1
ASIANPAINT.NS,3092.0,0.63,0.16,1.28,27.96,15.0,2954.17,87.9,71.6,H,19.0,1,1,1
RELIANCE.NS,2439.0,-1.36,0.08,0.43,37.96,15.0,2464.49,29.1,24.7,H,15.0,1,1,0
BAJFINANCE.NS,6705.0,1.53,0.47,0.86,24.47,18.0,6365.23,52.6,35.3,H,33.0,1,1,1
PAGEIND.NS,42396.0,-4.24,-0.23,1.11,31.29,27.0,39840.50,97.7,69.2,H,29.0,0,1,1


In [20]:
df_prec_dev.drop([ 'Avg_Cost', 'Shares', 'Med_PE', 'Curr_PE', 'Discount%PE'], axis=1, inplace=True)

In [21]:
# star + v80 stocks (for upside)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['v80'] == 1)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 5


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
BERGEPAINT.NS,620.0,2.50,0.50,1.01,858.0,533.0,73.23,38.0,1.0,0.0,M,1,1,1
UJJIVANSFB.NS,34.0,29.62,3.85,3.70,42.0,13.0,27.59,24.0,0.0,1.0,M,1,1,1
DABUR.NS,524.0,-5.08,-0.36,-0.19,645.0,464.0,66.85,23.0,1.0,0.0,H,1,1,1
OFSS.NS,3561.0,19.89,0.84,1.83,4392.0,2442.0,42.62,23.0,1.0,0.0,M,1,1,1
TCS.NS,3209.0,-0.38,0.37,-0.37,3886.0,2132.0,38.60,21.0,1.0,0.0,H,1,1,1


In [22]:
# top 5 from star + v80 for SIP
tmp_df.sort_values(by = 'Dev%_200', ascending=True).head(5)

,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
DABUR.NS,524.0,-5.08,-0.36,-0.19,645.0,464.0,66.85,23.0,1.0,0.0,H,1,1,1
TCS.NS,3209.0,-0.38,0.37,-0.37,3886.0,2132.0,38.60,21.0,1.0,0.0,H,1,1,1
BERGEPAINT.NS,620.0,2.50,0.50,1.01,858.0,533.0,73.23,38.0,1.0,0.0,M,1,1,1
OFSS.NS,3561.0,19.89,0.84,1.83,4392.0,2442.0,42.62,23.0,1.0,0.0,M,1,1,1
UJJIVANSFB.NS,34.0,29.62,3.85,3.70,42.0,13.0,27.59,24.0,0.0,1.0,M,1,1,1


In [23]:
# star + v80 stocks (for breakout)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['v80'] == 1)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 9


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
HDFCBANK.NS,1638.0,5.71,0.97,0.43,1708.0,1003.0,9.93,4.0,1.0,0.0,H,1,1,1
HINDUNILVR.NS,2662.0,3.65,0.12,0.68,2728.0,1897.0,7.94,2.0,1.0,1.0,H,1,1,1
CERA.NS,7353.0,30.33,2.38,1.19,7439.0,2229.0,1.65,1.0,1.0,1.0,H,1,1,1
ICICIBANK.NS,939.0,6.38,1.02,1.13,953.0,333.0,2.26,1.0,1.0,1.0,H,1,1,1
ITC.NS,428.0,22.17,2.00,1.25,431.0,146.0,1.05,1.0,1.0,1.0,H,0,1,1
TITAN.NS,2762.0,8.41,1.06,1.48,2790.0,1090.0,1.65,1.0,1.0,1.0,H,1,1,1
BRITANNIA.NS,4632.0,13.33,1.13,0.58,4635.0,2945.0,0.18,0.0,1.0,1.0,H,1,1,1
EQUITASBNK.NS,82.0,43.95,3.51,1.43,82.0,33.0,0.00,0.0,1.0,1.0,M,1,1,1
POLYCAB.NS,3438.0,24.64,1.99,1.07,3438.0,793.0,0.00,0.0,1.0,0.0,M,1,1,1


In [24]:
# star stocks (for upside)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['v80'] != 1)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 12


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
ADANIGREEN.NS,875.0,-47.07,-3.75,4.56,2970.0,377.0,80.79,239.0,-1.0,-1.0,L,1,1,0
ADANIENT.NS,1900.0,-34.48,-0.86,3.27,4165.0,233.0,57.60,119.0,1.0,0.0,M,1,1,0
BANDHANBNK.NS,244.0,-0.57,-0.82,0.46,422.0,187.0,75.74,73.0,1.0,1.0,L,0,1,0
LTIM.NS,4740.0,3.42,0.55,-0.22,7439.0,2267.0,52.18,57.0,1.0,0.0,M,1,1,0
GODREJIND.NS,468.0,6.18,0.23,1.16,644.0,354.0,60.69,38.0,0.0,1.0,L,0,1,0
TATAPOWER.NS,208.0,-3.37,-0.47,0.00,287.0,49.0,33.19,38.0,1.0,0.0,M,1,1,0
BAJAJFINSV.NS,1415.0,-6.39,0.60,0.53,1907.0,542.0,36.04,35.0,0.0,0.0,M,1,1,0
SBICARD.NS,879.0,7.37,0.00,1.70,1133.0,673.0,55.22,29.0,1.0,0.0,M,1,1,0
CIPLA.NS,924.0,-10.12,-0.10,-0.22,1173.0,701.0,52.75,27.0,1.0,0.0,M,1,1,0


In [25]:
# star stocks (for breakout)

tmp_df = df_prec_dev[df_prec_dev.index.isin(star_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
tmp_df = tmp_df[(tmp_df['Upside%ATH'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=True)

qualified stocks: 12


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
AUBANK.NS,731.0,16.01,0.95,2.05,731.0,317.0,0.00,0.0,1.0,0.0,M,1,1,0
BRITANNIA.NS,4632.0,13.33,1.13,0.58,4635.0,2945.0,0.18,0.0,1.0,1.0,H,1,1,1
EQUITASBNK.NS,82.0,43.95,3.51,1.43,82.0,33.0,0.00,0.0,1.0,1.0,M,1,1,1
POLYCAB.NS,3438.0,24.64,1.99,1.07,3438.0,793.0,0.00,0.0,1.0,0.0,M,1,1,1
CERA.NS,7353.0,30.33,2.38,1.19,7439.0,2229.0,1.65,1.0,1.0,1.0,H,1,1,1
ICICIBANK.NS,939.0,6.38,1.02,1.13,953.0,333.0,2.26,1.0,1.0,1.0,H,1,1,1
ITC.NS,428.0,22.17,2.00,1.25,431.0,146.0,1.05,1.0,1.0,1.0,H,0,1,1
TITAN.NS,2762.0,8.41,1.06,1.48,2790.0,1090.0,1.65,1.0,1.0,1.0,H,1,1,1
HINDUNILVR.NS,2662.0,3.65,0.12,0.68,2728.0,1897.0,7.94,2.0,1.0,1.0,H,1,1,1


In [26]:
# stocks excluded from all time high

excluded_stocks = ['ACC.NS','AMBUJACEM.NS','CADILAHC.NS','GILLETTE.NS','NESTLEIND.NS','PGHH.NS', 'PVRINOX.NS', 'SIEMENS.NS']

tmp_df = df_prec_dev[df_prec_dev.index.isin(excluded_stocks)]
tmp_df = tmp_df[~tmp_df.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
#tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] != -1)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 25) & (tmp_df['Discount%'] >= 60)]
#tmp_df = tmp_df[(tmp_df['Upside%ATH'] >= 20)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 5


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
ACC.NS,1758.0,-19.28,-0.92,0.29,2748.0,1259.0,66.49,56.0,0.0,0.0,L,0,0,0
PGHH.NS,13840.0,-0.71,0.01,0.15,15778.0,9190.0,29.42,14.0,0.0,0.0,L,0,0,1
PVRINOX.NS,1367.0,-5.43,NaN,-0.76,1483.0,1367.0,100.00,8.0,1.0,-1.0,L,0,0,0
NESTLEIND.NS,21732.0,11.39,0.91,1.63,22050.0,14623.0,4.28,1.0,1.0,1.0,H,0,0,1
SIEMENS.NS,3891.0,28.98,1.79,1.82,3891.0,1143.0,0.00,0.0,1.0,1.0,H,0,0,0


In [27]:
# prospects with heavy upside
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 2


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
INDUSTOWER.NS,151.0,-15.86,-1.67,-1.34,303.0,137.0,91.57,101.0,1.0,1.0,H,1,0,0
BANDHANBNK.NS,244.0,-0.57,-0.82,0.46,422.0,187.0,75.74,73.0,1.0,1.0,L,0,1,0


In [28]:
# prospects near 52W low
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] > 60]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=False)

qualified stocks: 3


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
INDUSTOWER.NS,151.0,-15.86,-1.67,-1.34,303.0,137.0,91.57,101.0,1.0,1.0,H,1,0,0
BANDHANBNK.NS,244.0,-0.57,-0.82,0.46,422.0,187.0,75.74,73.0,1.0,1.0,L,0,1,0
HDFCLIFE.NS,560.0,3.15,0.18,1.55,757.0,461.0,66.55,35.0,1.0,1.0,H,1,0,0


In [29]:
# prospects near 52W high
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[tmp_df['Discount%'] < 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Discount%', ascending=True)

qualified stocks: 14


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
EQUITASBNK.NS,82.0,43.95,3.51,1.43,82.0,33.0,0.00,0.0,1.0,1.0,M,1,1,1
SIEMENS.NS,3891.0,28.98,1.79,1.82,3891.0,1143.0,0.00,0.0,1.0,1.0,H,0,0,0
BRITANNIA.NS,4632.0,13.33,1.13,0.58,4635.0,2945.0,0.18,0.0,1.0,1.0,H,1,1,1
ITC.NS,428.0,22.17,2.00,1.25,431.0,146.0,1.05,1.0,1.0,1.0,H,0,1,1
CERA.NS,7353.0,30.33,2.38,1.19,7439.0,2229.0,1.65,1.0,1.0,1.0,H,1,1,1
TITAN.NS,2762.0,8.41,1.06,1.48,2790.0,1090.0,1.65,1.0,1.0,1.0,H,1,1,1
ICICIBANK.NS,939.0,6.38,1.02,1.13,953.0,333.0,2.26,1.0,1.0,1.0,H,1,1,1
INDUSINDBK.NS,1236.0,8.53,1.40,1.18,1264.0,484.0,3.59,2.0,1.0,1.0,H,1,1,0
NESTLEIND.NS,21732.0,11.39,0.91,1.63,22050.0,14623.0,4.28,1.0,1.0,1.0,H,0,0,1


In [30]:
# prospects in bull run
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 9


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
SIEMENS.NS,3891.0,28.98,1.79,1.82,3891.0,1143.0,0.00,0.0,1.0,1.0,H,0,0,0
NESTLEIND.NS,21732.0,11.39,0.91,1.63,22050.0,14623.0,4.28,1.0,1.0,1.0,H,0,0,1
HDFCLIFE.NS,560.0,3.15,0.18,1.55,757.0,461.0,66.55,35.0,1.0,1.0,H,1,0,0
TITAN.NS,2762.0,8.41,1.06,1.48,2790.0,1090.0,1.65,1.0,1.0,1.0,H,1,1,1
COLPAL.NS,1663.0,7.25,0.19,1.23,1741.0,1262.0,16.28,5.0,1.0,1.0,H,1,1,1
ICICIBANK.NS,939.0,6.38,1.02,1.13,953.0,333.0,2.26,1.0,1.0,1.0,H,1,1,1
HAVELLS.NS,1269.0,3.10,0.08,0.82,1461.0,606.0,22.46,15.0,1.0,1.0,H,1,0,1
HINDUNILVR.NS,2662.0,3.65,0.12,0.68,2728.0,1897.0,7.94,2.0,1.0,1.0,H,1,1,1
MARUTI.NS,9119.0,4.22,0.30,0.63,9528.0,6217.0,12.35,4.0,1.0,1.0,H,1,0,0


In [31]:
# prospects about to turn around
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] == 1) & (tmp_df['BizDurability'] == 1)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 2


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
BANDHANBNK.NS,244.0,-0.57,-0.82,0.46,422.0,187.0,75.74,73.0,1.0,1.0,L,0,1,0
INDUSTOWER.NS,151.0,-15.86,-1.67,-1.34,303.0,137.0,91.57,101.0,1.0,1.0,H,1,0,0


In [32]:
# cross-check the features
tmp_df = df_prec_dev[~df_prec_dev.index.isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['BizOutlook'] != 1) | (tmp_df['BizDurability'] != 1)]
tmp_df = tmp_df[tmp_df['Upside%ATH'] > 40]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%ATH', ascending=False)

qualified stocks: 47


,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
ADANITRANS.NS,790.0,-66.93,-4.19,1.37,4105.0,235.0,85.66,420.0,1.0,-1.0,L,0,0,0
PEL.NS,746.0,-21.10,-5.19,-0.70,2866.0,635.0,95.02,284.0,-1.0,0.0,L,1,0,0
IBREALEST.NS,56.0,-22.43,-1.39,0.00,191.0,47.0,93.75,241.0,-1.0,-1.0,L,0,0,0
ADANIGREEN.NS,875.0,-47.07,-3.75,4.56,2970.0,377.0,80.79,239.0,-1.0,-1.0,L,1,1,0
JCHAC.NS,1099.0,-11.51,-1.85,-1.07,2837.0,1017.0,95.49,158.0,NaN,NaN,M,0,0,1
TEAMLEASE.NS,2316.0,-13.43,-2.65,-1.15,5281.0,2028.0,91.15,128.0,NaN,NaN,M,0,0,1
ADANIENT.NS,1900.0,-34.48,-0.86,3.27,4165.0,233.0,57.60,119.0,1.0,0.0,M,1,1,0
LALPATHLAB.NS,2014.0,-9.38,-0.18,0.32,4186.0,1748.0,89.09,108.0,NaN,NaN,M,1,0,1
WHIRLPOOL.NS,1348.0,-10.48,-1.13,0.38,2667.0,1254.0,93.35,98.0,1.0,0.0,M,0,0,1


In [33]:
# analyse individual stock
stock_id = 'PVRINOX.NS'
df_prec_dev[df_prec_dev.index == stock_id]

,Close,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH,BizOutlook,BizDurability,Conviction,LatestQtr,StarStock,v80
Stock,,,,,,,,,,,,,,
PVRINOX.NS,1367.0,-5.43,NaN,-0.76,1483.0,1367.0,100.0,8.0,1.0,-1.0,L,0,0,0


In [34]:
# save the output
#df_prec_dev = df_prec_dev.reset_index()
#df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/Nifty100-Upside-Analysis-ATH.csv', index=False)

In [35]:
# nifty in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BHARTIARTL.NS')[-21::5]

,Close,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Discount%,Upside%ATH
Date,,,,,,,,,
2023-04-18 00:00:00+05:30,760.0,BHARTIARTL.NS,-1.07,0.78,0.00,851.0,396.0,20.00,12.0
2023-04-25 00:00:00+05:30,773.0,BHARTIARTL.NS,0.34,0.65,-0.26,851.0,396.0,17.14,10.0
2023-05-03 00:00:00+05:30,779.0,BHARTIARTL.NS,0.95,0.52,0.26,851.0,396.0,15.82,9.0
2023-05-10 00:00:00+05:30,796.0,BHARTIARTL.NS,2.58,0.77,0.13,851.0,396.0,12.09,7.0
2023-05-17 00:00:00+05:30,792.0,BHARTIARTL.NS,1.70,0.90,0.39,851.0,396.0,12.97,7.0
